In [ ]:
import pandas as pd
import numpy as np

from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pylab import rcParams
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
# mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Helvetica']

# plt.style.use('seaborn-white') # use sans-serif fonts
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

rcParams['figure.figsize'] = 5, 5

mpl.rcParams['axes.spines.left'] = True   ## display axis spines
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['ytick.major.width'] = 1.0

import henonheiles
import importlib
importlib.reload(henonheiles)
import henonheiles as HH2dof

params = [1.0, 1.0, 1.0, 1.0, 1.0]

In [ ]:
# %matplotlib

import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# training_data_size = 625
training_data_size = 10000
total_energy = 0.17
trainingdata_time = 30

manifold_time = 80
datapath_ris = '../../henonheiles/data/manifolds_E17e-2/firstRIs/'
# datapath_ris = '../../henonheiles/data/manifolds_E17e-2/firstRIs_sos_y-0.25/'
# datapath_ris = '../../henonheiles/data/manifolds_E19e-2/firstRIs_sos_y0.5/'

# datapath_mlmodels = './hh2dof_firstRIs/fixed_trainingdata_size/'
# datapath_mlmodels = '../data/hh2dof_firstRIs/adaptive_trainingdata_size/'
# datapath_mlmodels = './hh2dof_firstRIs_sos_y-0.25/adaptive_trainingdata_size/'
# datapath_mlmodels = '../data/hh2dof_firstRIs_sos_y-0.25/svcr/'
datapath_mlmodels = '../data/hh2dof_firstRIs/svcr/'
# datapath_mlmodels = '../data/hh2dof_firstRIs_sos_y0.5/svcr/'
# datapath_mlmodels = './hh2dof_firstRIs_sos_y-0.25/fixed_trainingdata_size/'
# datapath_mlmodels = '../data/hh2dof_firstRIs_sos_y0.5/fixed_trainingdata_size/'
# datapath_mlmodels = './hh2dof_firstRIs_sos_y0.5/adaptive_trainingdata_size/'

y_constant = 0
# y_constant = -0.25
# y_constant = 0.5

ri_topsaddle, ri_leftsaddle, ri_rightsaddle = HH2dof.get_ris_data(datapath_ris, total_energy, \
                                                                 manifold_time)

energy_boundary = henonheiles.energy_boundary_sos_xpx(params, total_energy, y_constant)

# data = pd.read_csv(datapath_mlmodels + 'hh_escape_samples' \
#                    + str(training_data_size) + '_E%.3f'%(total_energy) \
#                    + '_T%.3f'%(trainingdata_time) + '.txt', \
#                    sep = " ", header = None, \
#                    names = ["x", "y", "p_x", "p_y", "Exit channel"])

data = pd.read_csv(datapath_mlmodels + 'hh_escape_samples%d'%(training_data_size) \
                   + '_E%.3f'%(total_energy) \
                   + '_T%.3f'%(trainingdata_time) + '.txt', \
                   sep = " ", header = None, \
                   names = ["x", "y", "p_x", "p_y", "TE", "LD", "Exit channel"])

#print(data.shape,'\n',data.head)

data = data.dropna()
# print('Samples on the energy surface:%d'%(data.shape[0]),'\n',data.head)


X = data.drop(["y", "p_y", "TE", "Exit channel"], axis=1)
# X = data.drop(["y", "p_y", "Exit channel"], axis=1)
# X = data.drop(["Exit channel"], axis=1)
y = data["Exit channel"].astype(int)

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size=0.2, random_state=0)


clf_model = joblib.load(datapath_mlmodels \
                        + 'hh_svc_samples' + str(training_data_size) + '_E%.3f'%(total_energy) \
                        + '_T%.3f'%(trainingdata_time) + '.sav')
result = clf_model.best_estimator_.score(X_test, y_test)
print(result)

# support_vectors = scaler.inverse_transform(clf_model.best_estimator_.support_vectors_)
support_vectors = clf_model.best_estimator_.support_vectors_
print('Number of support vectors: %d'%(len(support_vectors)))

# plt.scatter(clf_model.support_vectors_[:,0], clf_model.support_vectors_[:,1], \
#            s = 2, c = 'g')

fig_mani_sect = plt.figure(figsize = (10,10))
ax_mani_sect = fig_mani_sect.gca()
plt.plot(energy_boundary[:,0], energy_boundary[:,1], '-m', linewidth = 2)
plt.plot(ri_topsaddle[:,1], ri_topsaddle[:,3], '.r', markersize = 2)
plt.plot(ri_leftsaddle[:,1], ri_leftsaddle[:,3],'.g', markersize = 2)
plt.plot(ri_rightsaddle[:,1], ri_rightsaddle[:,3],'.b', markersize = 2)

# ax_mani_sect.set_xlim([xMin_at_yconstant, xMax_at_yconstant])
# ax_mani_sect.set_ylim([pxMin_at_yconstant, pxMax_at_yconstant])
# ax_mani_sect.set_xlim([-0.6, 0.6])
# ax_mani_sect.set_ylim([-0.6, 0.6])

plt.scatter(support_vectors[:,0], support_vectors[:,1], s = 10, c = 'cyan')
ax_mani_sect.set_ylabel(r'$p_x$', labelpad = 5, rotation = 0, fontsize = 30)
ax_mani_sect.set_xlabel(r'$x$', labelpad = 0, fontsize = 30)
plt.savefig('supportvectors_RIs' + '_E%.3f'%(total_energy) + '.png', \
            dpi = 300, bbox_inches = 'tight')



In [ ]:
clf_model.best_estimator_.score(X_test, y_test)

In [ ]:
plt.plot(y_train, '.k')

## Plotting the validation accuracy for hyper-parameter optimization

In [ ]:
C_range = clf_model.param_grid[0]['C']
gamma_range = clf_model.param_grid[0]['gamma']
scores = clf_model.cv_results_['mean_test_score'].reshape(len(C_range), len(gamma_range))

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar(shrink = 0.5)
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('Validation accuracy')
plt.show()

In [ ]:
clf_model

In [ ]:
from sklearn.svm import SVR

# X = data.drop(["y", "p_y", "TE", "LD", "Exit channel"], axis=1)
# X = data.drop(["TE", "LD", "Exit channel"], axis=1)
# y = data["LD"]

X = data.drop(["y", "p_y", "Exit channel"], axis=1)
# X = data.drop(["Exit channel"], axis=1)
y = data["Exit channel"].astype(int)



X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size=0.2, random_state=0)

svr_rbf = SVR(kernel='rbf', C = 100, gamma = 10, epsilon = .05)

svr_rbf_fit = svr_rbf.fit(X_train, y_train)

In [ ]:
import numpy as np
import henonheiles
import importlib
importlib.reload(henonheiles)
import henonheiles as HH2dof

res = 300
xMax_at_yconstant = np.sqrt((total_energy - \
                     (0.5*params[3]**2*y_constant**2 - \
                      (params[4]/3)*y_constant**3))/(0.5*params[2]**2 + y_constant))
xMin_at_yconstant = -xMax_at_yconstant

pxMax_at_yconstant = np.sqrt(2*params[0]*(total_energy - HH2dof.potential_energy(0,y_constant, params[2:])))
pxMin_at_yconstant = -pxMax_at_yconstant


px_at_yconstant = lambda x: np.sqrt(2*params[0]*(total_energy - \
                                    (0.5*params[2]**2*x**2 + 0.5*params[3]**2*y_constant**2 - \
                                     (params[4]/3.0)*y_constant**3 + x**2*y_constant) ))


xGrid_boundary = np.linspace(xMin_at_yconstant + 1e-10, xMax_at_yconstant - 1e-10, 401, endpoint = True)
pxGrid_boundary = px_at_yconstant(xGrid_boundary)
pxGrid_boundary[0] = 0
pxGrid_boundary[-1] = 0

xGrid_boundary = np.append(xGrid_boundary, np.flip(xGrid_boundary))
pxGrid_boundary = np.append(pxGrid_boundary, -np.flip(pxGrid_boundary))


xMesh, pxMesh = np.meshgrid(np.linspace(xMin_at_yconstant + 1e-6, xMax_at_yconstant - 1e-6, res), \
                           np.linspace(pxMin_at_yconstant + 1e-6, pxMax_at_yconstant - 1e-6, res))

        
X_grid = np.column_stack([xMesh.ravel(), pxMesh.ravel()])
# LD_predict = svr_rbf_fit.predict(X_grid)
# X_grid = np.column_stack([X_grid, LD_predict])
# scaler = StandardScaler()
# X_grid = scaler.fit_transform(X_grid)

# decision_grid = clf_model.best_estimator_.decision_function(X_grid)
decision_grid = clf_model.best_estimator_.predict(X_grid)

# X_grid = scaler.inverse_transform(X_grid)

fig, ax = plt.subplots(figsize = (10,10))
# plt.scatter(xMesh, pxMesh, s = 0.01)
# plt.scatter(X_grid[:,0],X_grid[:,1], s = 0.01)
plt.plot(xGrid_boundary, pxGrid_boundary, '-m', linewidth = 2)

plt.contour(xMesh, pxMesh, decision_grid.reshape(res,res), \
            levels = [0],linewidths = 3, \
            linestyles='dashed', colors='black')
# plt.contourf(xMesh, pxMesh, LD_predict.reshape(res,res), 100)

# plt.plot(smani_topsaddle[:,1], smani_topsaddle[:,3], '.r', markersize = 2)
# plt.plot(smani_rightsaddle[:,1], smani_rightsaddle[:,3],'.b', markersize = 2)
# plt.plot(smani_leftsaddle[:,1], smani_leftsaddle[:,3],'.g', markersize = 2)

plt.plot(ri_topsaddle[:,1], ri_topsaddle[:,3], '.r', markersize = 2)
plt.plot(ri_leftsaddle[:,1], ri_leftsaddle[:,3],'.g', markersize = 2)
plt.plot(ri_rightsaddle[:,1], ri_rightsaddle[:,3],'.b', markersize = 2)



plt.scatter(support_vectors[:,0], support_vectors[:,1], s = 10, c = 'cyan')

# ax.set_xlim([xMin_at_yconstant, xMax_at_yconstant])
# ax.set_ylim([pxMin_at_yconstant, pxMax_at_yconstant])

ax.set_ylabel(r'$p_x$', labelpad = 5, rotation = 0, fontsize = 20)
ax.set_xlabel(r'$x$', labelpad = 0, fontsize = 20)
# plt.savefig('decision_boundaries_supportvectors_RIs' + '_E%.3f'%(total_energy) + '.png', \
#             dpi = 300, bbox_inches = 'tight')


# plt.contour(xMesh, yMesh, decision_grid[:,1].reshape((res, res)),cmap=plt.cm.coolwarm, alpha=0.8)
# plt.contour(xMesh, yMesh, decision_grid[:,2].reshape((res, res)),cmap=plt.cm.coolwarm, alpha=0.8)
# plt.contour(xMesh, yMesh, decision_grid[:,3].reshape((res, res)),cmap=plt.cm.coolwarm, alpha=0.8)

# contours = plt.contour(xMesh, yMesh, decision_grid_left, levels=[0], linewidths=2, \
#                        linestyles='dashed', colors='green')
# contours = plt.contour(xGrid, pxGrid, decision_grid_right, levels=[0], linewidths=2, \
#                        linestyles='dashed', colors='blue')
# contours = plt.contour(xGrid, pxGrid, decision_grid_top, levels=[0], linewidths=2, \
#                        linestyles='dashed', colors='red')



In [ ]:
plt.plot(decision_grid)


In [ ]:
pip install --upgrade pandas